Train a classification model for the SketchRNN dataset, available in TensorFlow Datasets.

In [2]:
import tensorrt
import tensorflow as tf
import tensorflow_datasets as tfds
from pathlib import Path

In [3]:
tf_download_root = "http://download.tensorflow.org/data/"
filename = "quickdraw_tutorial_dataset_v1.tar.gz"
filepath = tf.keras.utils.get_file(filename,
                                   tf_download_root + filename,
                                   cache_dir=".",
                                   extract=True)

In [4]:
def parse(data_batch):
    feature_descriptions = {
        "ink": tf.io.VarLenFeature(dtype=tf.float32),
        "shape": tf.io.FixedLenFeature([2], dtype=tf.int64),
        "class_index": tf.io.FixedLenFeature([1], dtype=tf.int64)
    }
    examples = tf.io.parse_example(data_batch, feature_descriptions)
    flat_sketches = tf.sparse.to_dense(examples["ink"])
    sketches = tf.reshape(flat_sketches, shape=[tf.size(data_batch), -1, 3])
    lengths = examples["shape"][:, 0]
    labels = examples["class_index"][:, 0]
    # return sketches, lengths, labels
    return sketches, labels

In [5]:
def extract_labels(file_path):
    with open(file_path) as f:
        return [x for x in f.read().split("\n") if x]

In [6]:
def crop_long_sketches(dataset, max_length=100):
    return dataset.map(lambda inks, labels: (inks[:, :max_length], labels))

In [7]:
root_dir = Path('datasets')
training_files = list(root_dir.glob('training.tfrecord-*'))
eval_files = list(root_dir.glob('eval.tfrecord-*'))
# Length: 3,450,000
training_ds = tf.data.TFRecordDataset(training_files, num_parallel_reads=5)
# Length: 345,000
validation_ds = tf.data.TFRecordDataset(eval_files[:5], num_parallel_reads=5)
test_ds = tf.data.TFRecordDataset(eval_files[5:], num_parallel_reads=5)

training_ds = crop_long_sketches(training_ds.shuffle(10000).take(100000).batch(32).map(parse, num_parallel_calls=5).prefetch(1))
validation_ds = crop_long_sketches(validation_ds.batch(32).map(parse, num_parallel_calls=5).prefetch(1))
test_ds = crop_long_sketches(test_ds.batch(32).map(parse, num_parallel_calls=5).prefetch(1))

# training_labels = extract_labels(root_dir / 'training.tfrecord.classes')
# eval_labels = extract_labels(root_dir / 'eval.tfrecord.classes')
# training_labels == eval_labels
# # => True

labels = extract_labels(root_dir / 'training.tfrecord.classes')
print(len(labels))

2023-07-06 14:23:52.523491: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-06 14:23:52.546154: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-06 14:23:52.546196: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-06 14:23:52.548917: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-06 14:23:52.548958: I tensorflow/compile

345


In [8]:
if False:
    for sketches, lengths, labels in training_ds:
        # batch size, sequence length, 3
        # Note that zero padding is applied in the end when the sequence is too short
        print(sketches.shape)
        # print(sketches[0])
        print(lengths)
        print(labels)
        print(sketches[0])
        break

In [9]:
for x in training_ds:
    print(x[1].shape)
    break

(32,)


2023-07-06 14:23:53.576742: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [10]
	 [[{{node Placeholder/_0}}]]
2023-07-06 14:23:53.576990: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [10]
	 [[{{node Placeholder/_0}}]]


In [10]:
model = tf.keras.Sequential([
    # tf.keras.layers.Input(shape=[None, 3]),
    tf.keras.layers.Conv1D(32, kernel_size=5, strides=2, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(64, kernel_size=5, strides=2, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(128, kernel_size=3, strides=2, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Masking(mask_value=0.0),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(345, activation='softmax')
])
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2, clipnorm=1.),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

In [11]:
model_root_dir = Path() / 'data' / '01-sketch-rnn'
tensorboard_dir = model_root_dir / 'tensorboard'
print(f'tensorboard --logdir=ch15/{tensorboard_dir}')
checkpoints_dir = model_root_dir / 'checkpoints'
checkpoints_cb = tf.keras.callbacks.ModelCheckpoint(checkpoints_dir, save_best_only=True)
tensorboard_cb = tf.keras.callbacks.TensorBoard(tensorboard_dir)

tensorboard --logdir=ch15/data/01-sketch-rnn/tensorboard


In [12]:
hist = model.fit(
    training_ds,
    epochs=2,
    validation_data=validation_ds,
    callbacks=[tensorboard_cb, checkpoints_cb]
)

Epoch 1/2


2023-07-06 14:23:54.195827: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-06 14:23:54.197485: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-06 14:23:54.198648: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

   3122/Unknown - 63s 17ms/step - loss: 5.7578 - sparse_categorical_accuracy: 0.0055

2023-07-06 14:24:57.404776: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [5]
	 [[{{node Placeholder/_0}}]]
2023-07-06 14:24:57.405008: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [5]
	 [[{{node Placeholder/_0}}]]
2023-07-06 14:24:57.595400: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with d

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(5, 3, 32), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f464cd59bd0>, 139940843864832), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(32,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f46401fbe10>, 139939915732528), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(32,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f46401e5bd0>, 139940844019808), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(32,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f464020d510>, 139940844020208), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(5, 32, 64), dtype=

...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(345,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f4640097050>, 139939913084464), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(345,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f4640097050>, 139939913084464), {}).


INFO:tensorflow:Assets written to: data/01-sketch-rnn/checkpoints/assets


INFO:tensorflow:Assets written to: data/01-sketch-rnn/checkpoints/assets


3125/3125 [==============================] - 74s 24ms/step - loss: 5.5646 - sparse_categorical_accuracy: 0.0104 - val_loss: 5.5128 - val_sparse_categorical_accuracy: 0.0152
